In [13]:
import openpyxl as pyxl
import pandas as pd
from itertools import islice
excel_report = pyxl.load_workbook("./mocks/excel_reports/10-K-20.xlsx")


In [14]:
##Code here works want to keep it.

sheets_name_series = pd.Series(excel_report.sheetnames)
pattern = '^Condensed|^Consolidated|^CONSOLIDATED|^CONDENSED|^condensed|^consolidated'
to_keep = sheets_name_series.str.contains(pattern)
to_drop = (sheets_name_series[~to_keep])[1:]
for sheet_name in to_drop:
    excel_report.remove(excel_report.get_sheet_by_name(sheet_name))

<ipython-input-14-ebc0e1683fc9>:8: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  excel_report.remove(excel_report.get_sheet_by_name(sheet_name))


In [15]:
## gonna try this again but also try to concat the cell below the merged column

for sheets in excel_report.worksheets:
    while sheets.merged_cells.ranges:
        range = str(sheets.merged_cells.ranges[0])
        sheets.unmerge_cells(range)
        if len(sheets[range]) == 1:
            for col_cell in sheets[range][0]:
                # haven't seen a spreadsheet yet where this isnt the case for merged cells at the top.
                next_coord = col_cell.coordinate[0] + '2'
                sheets[next_coord].value = sheets[next_coord].value + ' - ' + sheets[range][0][0].value
            sheets[range][0][0].value = None

## Notes: the merged_cells.ranges returns a range... then access those cells in sheets[range]
## returns a tuple of tuples. if it is all the same column then it is n-tuple. if it is range
## columns then it is a 1-tuple.
## https://openpyxl.readthedocs.io/en/stable/tutorial.html

## Unmerged cells, work and saw the output.
for sheets in sheets_dict.values():
    while sheets.merged_cells.ranges:
        range = str(sheets.merged_cells.ranges[0])
        sheets.unmerge_cells(range)
        if len(sheets[range]) == 1:
            for col_cell in sheets[range][0]:
                col_cell.value = sheets[range][0][0].value

In [16]:
## note when saving back into excel must rename the sheet names with a 31 character limit.
## thus will have to undo this but it is necessary for pandas

for sheets in excel_report.worksheets:
    to_cut = sheets['A1'].value.index('-')
    sheets.title = sheets['A1'].value[:to_cut-1]
    sheets['A1'].value = sheets['A1'].value[to_cut+2:]

print(excel_report.sheetnames)

['Document And Entity Information', 'Consolidated Balance Sheets', 'Consolidated Balance Sheets (Parentheticals)', 'Consolidated Statements of Operations', 'Consolidated Statements of Comprehensive Income (Loss)', 'Consolidated Statements of Cash Flows', "Consolidated Statements of Stockholders' Equity", "Consolidated Statements of Stockholders' Equity (Parentheticals)"]


C:\Users\Tutrie\anaconda3\envs\CMSC435\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [17]:
sheets_dict = {}
for sheets in excel_report.worksheets:
    notes = sheets['A1'].value
    sheets['A1'].value = None
    if sheets['B1'].value == None:
        sheets.delete_rows(1)
    data = sheets.values
    cols = next(data)[1:]
    data = list(data)
    idx = [r[0] for r in data]
    data = (islice(r, 1, None) for r in data)
    sheets_dict[sheets.title] = (pd.DataFrame(data, index=idx, columns=cols), notes)

excel_report.save('test.xlsx')

In [18]:
for df in sheets_dict.values():
    df
    break

In [19]:
list(sheets_dict.values())[0]

(                                                              12 Months Ended  \
 NaN                                                             Nov. 28, 2020   
 Document Information [Line Items]                                        None   
 Entity Central Index Key                                           0000010329   
 Entity Registrant Name                       BASSETT FURNITURE INDUSTRIES INC   
 Amendment Flag                                                          false   
 Current Fiscal Year End Date                                          --11-28   
 Document Fiscal Period Focus                                               FY   
 Document Fiscal Year Focus                                               2020   
 Document Type                                                            10-K   
 Document Annual Report                                                   true   
 Document Period End Date                                   Nov. 28,\n\t\t2020   
 Document Transi